# 1. Introduction

- I decided to go with approach 2 "Share and grab data". I took 3 other students images which increased my original training images amount by 793 and my original validation images by 224. (total of 1053 training images and 322 validation images)
- I chose this approach because by combining my images with other students I created a larger dataset. In theory, this should produce a better model by allowing the model to learn a better approximation (which will increase production performance). 
- I implemented this approach by:
    - downloading the images into their respective digit folders on my google drive to combine.
    - retraining the model with a larger dataset in lab 3 
    - aquiring 5 examples of each digit in lab 4 and computed accuracy and confusion matrix again for produciton performance

# 2. Model training and Results


From Lab 3
- created a datablock and dataloaders with aug_transforms the same as in lab 3
- using a resnet 18 architecture looked at the learning rate/loss graph
- ![title](lr.png)
- chose a lr of 9e-4 accuracy improved to 0.67 compared to out of the box of 0.37
- Performed transfer learning, unfroze and found lr with different landscape (lr = 7e-4)
- ![title](lr1.png)
- Used discriminative learning rates (9e-5,9e-3)
- Played with different batch sizes, 16 and 64, found that the ideal batch size is 16 with 12 epochs
- Retrained on the ideal batch size producing the following confusion matrix:
- ![title](confmat.png)
- saved the best model to file

In [ ]:
# lab 3 code done in google colab 
!pip install -q fastbook

import fastbook
fastbook.setup_book()
from fastai.vision.all import *

In [ ]:
path = Path('/content/gdrive/MyDrive/Colab Notebooks/digits')

digits = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items = get_image_files, 
                 splitter = set_seed(42),
                 get_y = parent_label,
                 item_tfms = Resize(640, 'squish'),
                 batch_tfms = aug_transforms(size=420, max_rotate = 90, max_zoom = 0.25, pad_mode= 'border'))
dls = digits.dataloaders(path, bs = 32)


In [ ]:
learn = cnn_learner(dls, resnet18, metrics=accuracy)
lr_min, lr_steep, lr_valley, lr_slide = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
learn.fine_tune(1, base_lr=9e-4)

In [ ]:
# transfer learning
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fit_one_cycle(2, 9e-4)

In [ ]:
# unfreeze network and find learning rate again with different landscape
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(4, lr_max=7e-4)
# performance improves

In [ ]:
# discrimnative learning rates
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fit_one_cycle(2, 9e-4)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(9e-5,9e-3))

In [ ]:
# batch size of 16
dls = digits.dataloaders(path, bs = 16)
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fit_one_cycle(2, 9e-4)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(9e-5,9e-3))

In [ ]:
# batch size of 64
dls = digits.dataloaders(path, bs = 64)
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fit_one_cycle(2, 9e-4)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(9e-5,9e-3))

In [ ]:
# retraining on ideal bs
dls = digits.dataloaders(path, bs = 16)
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fit_one_cycle(2, 9e-4)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(9e-5,9e-3))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=1)

In [ ]:
export_path = Path('/content/gdrive/MyDrive/Colab Notebooks')

if not (export_path/'models').exists():
  (export_path/'models').mkdir()

In [ ]:
learn.export(fname=(export_path/"models/digits_modelNEW.pkl"))
(export_path/"models").ls()

From lab 4
- Saved 5 sets of labeled images (using lab4_predict_finger_count_gui.py) for each digit and created a csv with the results
- created a matrix with the results and calculated accuracy = 92%

In [1]:
# lab 4 code

import pandas as pd

test_df = pd.read_csv('lab4-results.csv')

actualValue = test_df['actual']
predictedValue = test_df['predicted']

y_actu = pd.Series(actualValue, name = "Actual")
y_pred = pd.Series(predictedValue, name = "Predicted")
df_confusion = pd.crosstab(y_actu, y_pred)
print(df_confusion)

Predicted  1  2  3  4  5
Actual                  
1          5  0  0  0  0
2          0  5  0  0  0
3          0  1  4  0  0
4          0  0  1  4  0
5          0  0  0  0  5


In [2]:
# ACCURACY:
correct = 5 + 5 + 4 + 4 + 5
total = correct + 1 + 1
accuracy = correct/total
print(f"Accuracy: {accuracy*100}%")

Accuracy: 92.0%


# 3. Summary and Conclusion
Summary of new results and lab3/lab4 results.
Lab 3 results:
- Training loss: 0.68
- Validation loss: 0.39
- Accuracy achieved = 84.8% (validation images)
- Three was predicted as four 4 times
- Four was predicted as three twice
- Two was predicted as three twice

Lab 4 results:
- Accuracy achieved = 84.0% (production images)
- four was predicted as three twice
- two was predicted as three once
- five was predicted as three once

New results:
- Training loss: 0.28
- Validation loss: 0.15
- Accuracy achieved = 95.2% (validation images)
- Accuracy achieved = 92.0% (production images)

Conclusion: 
    Due to the approach of using more data to train the model, the perfomance of the classifier improved by 8%. More data should produce a better model; by pulling more (quality) data from others (and augmenting these images) the model gets trained on more variations creating a more complex problem than before. Larger data sets to train on gives the model a chance to learn more features in the contrasting images. This should allow the classifier to reasonably approximate and predict unknown/unseen images. Increased data should often lead to more predictive power for the model.
    In this case the model was able to capture the complex data present. This was seen in the training progression where both the losses decreased and the accuracy got better with each epoch. When put into production it was immediatly clear that the model was performing better. Not only did the production accuracy increase but the probability of the correct predictions were all high. 


# 4. Reflection
Include a sentence or two about
- I really enjoyed the progression of these labs and how they built off each other. The final product is something to be proud of
- Being able to see your model working in real time was like magic. So cool
- I was almost hoping my model didnt perform better so I would have to do some digging into why that happened
- As always I really enjoyed this course. Thanks for another one Dr.Pauchard
